In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
INPUT_SHAPE = (256, 256)
CHANNEL = 3
MODEL_NAME = "model3"

# IMAGE_PATH = "./datasets/test/dora-over-armary-close.jpg"
# IMAGE_PATH = "./datasets/test/manydogs.jpeg"
# IMAGE_PATH = "./datasets/test/meow.jpg"
# IMAGE_PATH = "./datasets/test/panda.jpg"
IMAGE_PATH = "./datasets/test/dora_stares_at_you.jpg"

IMAGE_NAME = IMAGE_PATH.split("/")[3]

# Introduction

This procedure consist on obtaining how important the ontributions of each of the channels when finding the class the object belongs to.

With this procedure we are able to understand which parts of the images producce the most amount of activations

# load the models

In [ ]:
model = tf.keras.models.load_model(f"./checkpoints/{MODEL_NAME}/")

model = tf.keras.applications.ResNet50(include_top=True)
# model = tf.keras.applications.VGG19(include_top=True)

In [ ]:
model.summary()

In [ ]:
# TODO: llevar a lib aparte

def get_convs_layers(model):
    outputs=[]
    for layer in model.layers:
        if isinstance(layer, (tf.keras.layers.Conv2D, tf.keras.layers.MaxPool2D)):
            layer.trainable=False
            outputs.append(layer)
    return outputs

In [ ]:
last_layer_conv_output = get_convs_layers(model)[-1].output

In [ ]:
last_layer_conv_output.shape

In [ ]:
last_conv_model = tf.keras.Model(inputs=model.input, outputs=last_layer_conv_output)

In [ ]:
model.layers[-4:][0].output.shape

In [ ]:
# We get a model that will take the output of the last convolutional layer and produce a prediction
classification_model_input = tf.keras.Input(shape=last_conv_model.output.shape[1:])

x = classification_model_input

classification_layers = model.layers[-2:]
#Build the graph from the output of the last model to the classification layer model
for layer in classification_layers:
    x = layer(x)

classification_model_output = tf.keras.Model(inputs=classification_model_input, outputs=x)

In [ ]:
classification_model_output.summary()

In [ ]:
#load a sample image

INPUT_SHAPE = model.input.shape[1:-1]

image = tf.keras.utils.load_img(IMAGE_PATH, color_mode="rgb" if CHANNEL==3 else "grayscale", target_size=(INPUT_SHAPE))
image_vector = tf.keras.utils.img_to_array(image)

In [ ]:
image_vector/=256
plt.imshow(image_vector)

In [ ]:
image_vector = np.expand_dims(image_vector, axis=0)

In [ ]:
with tf.GradientTape() as tape:
    last_conv_activation = last_conv_model(image_vector)
    tape.watch(last_conv_activation)
    classfication_output = classification_model_output(last_conv_activation)
    top_pred_index = tf.argmax(classfication_output[0])
    top_class_channel = classfication_output[:, top_pred_index]

activation_gradient = tape.gradient(top_class_channel, last_conv_activation)

In [ ]:
# Gradient corresponding to the last convolutional layer when applied to the classification output

activation_gradient[0].shape

In [ ]:
# activation_gradient = activation_gradient[0].numpy()

In [ ]:
# Average on for channel
# average_gradient_activation = np.average(activation_gradient, axis=2)

In [ ]:
# average_gradient_activation.shape

In [ ]:
# pooling chanel importance

In [ ]:
pooled_grads = tf.reduce_mean(activation_gradient, axis=(0, 1, 2)).numpy()
last_conv_activation = last_conv_activation.numpy()[0]

In [ ]:
pooled_grads.shape

In [ ]:
# Weight the values from the last activation 
for i in range(pooled_grads.shape[-1]):
    last_conv_activation[:, :, i] *= pooled_grads[i]

#get the average value per channel
heathmap = np.mean(last_conv_activation, axis=-1)

#remove negative values
heathmap = np.maximum(heathmap, 0)

# scale values of the heatmap to keep it between 0 - 1
heathmap /= np.max(heathmap)

plt.imshow(heathmap)

In [ ]:
# import matplotlib.cm as cm

# jet = cm.get_cmap("jet")

# jet_colors = jet(np.arange(256))[:, :CHANNEL]

In [ ]:
import matplotlib.cm as cm

image = tf.keras.utils.load_img(IMAGE_PATH, color_mode="rgb" if CHANNEL==3 else "grayscale")
image = tf.keras.utils.img_to_array(image)

heathmap = np.uint(255 * heathmap)
jet = cm.get_cmap("jet")
jet_colors = jet(np.arange(256))[:, :CHANNEL]
jet_heatmap = jet_colors[heathmap]

jet_heatmap = tf.keras.utils.array_to_img(jet_heatmap)
jet_heatmap = jet_heatmap.resize((image.shape[1], image.shape[0]))
jet_heatmap = tf.keras.utils.img_to_array(jet_heatmap)

# Combine heatmal with the image superpose both
superimpose_img = jet_heatmap * 0.6 + image
superimpose_img = tf.keras.utils.array_to_img(superimpose_img)

save_path = f"./heatmaps-part3/{IMAGE_NAME}_{MODEL_NAME}.jpg"
superimpose_img.save(save_path)

plt.imshow(superimpose_img)